In [1]:
import pandas as pd
import sinta_scraper as sinta
import numpy as np

# Import search_author module as it is not yet available in sinta_scraper
import sys
import os
module_path = os.path.abspath(os.path.join('../scripts/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from author_search import search_author

import ast

In [2]:
univ = [i.lower() for i in os.listdir('../data') if not i.startswith('.')]

In [3]:
# get life sciences author from UGM
## Set variables
with open(f"../data/{univ[0].upper()}/ugm_affil_information.txt", "r", encoding = 'utf-8') as f:
    d = ast.literal_eval(f.read())
    
affil_name = d['affil_name']
affil_name_long = d['affil_name_long']
affil_id = d['affil_id']
dept_id = d['dept_id']

In [4]:
# get authors by dept_id
SINTA = pd.DataFrame()
for de in dept_id:
    data = sinta.dept_authors(de, affil_id)
    data = pd.DataFrame(data)
    #print(de, data)
    data.loc[:, 'department'] = de
    SINTA = SINTA.append(data)
#SINTA.to_csv(f'../tables/{affil_name}_bio_SINTA.csv')
#SINTA

In [5]:
# Load data in manual
MANUAL = pd.read_csv(f'../data/{affil_name.upper()}/data_NIDN-versi-sinta.txt', index_col=0, dtype=str)
#MANUAL

In [6]:
# Merge Manual and SINTA
MERGE = pd.merge(MANUAL, SINTA, on=["name", "id"], how='outer')
MERGE.to_csv(f'../tables/{affil_name}_bio_MERGE.csv')
#MERGE

In [7]:
# Filter retired faculty member
RETIRED = pd.read_csv(f'../data/{affil_name.upper()}/{affil_name}_bio_retired.txt')
mask = MERGE.name.isin(RETIRED.Retired)
ALL = MERGE[~mask]
#ALL

In [8]:
# first clean?
#ALL = ALL.sort_values(by=['name']).reset_index(drop=True)
#ALL.to_csv(f'../data/{affil_name.upper()}/all.csv')
#ALL

In [9]:
# Get Missing SINTA ID
missing_SINTAID = ALL[pd.isnull(ALL.id)]
nama = missing_SINTAID.name
output_missing_SINTAID = pd.DataFrame()
for i in nama:
    print(i)
    data = search_author(i)
    if data != []:
        data = pd.DataFrame(data)
        data = data[data.affiliation == affil_name_long]
        output_missing_SINTAID = output_missing_SINTAID.append(data)
    else:
        print(f'{i} not found in SINTA Database')
#output_missing_SINTAID

# Add missing SINTA id
for i in ALL[ALL.id.isnull()].index:
    x = ALL.loc[i, 'name'].upper()
    if output_missing_SINTAID.name.isin([x]).any():
        new_id = output_missing_SINTAID[output_missing_SINTAID.name.isin([x])].id
        ALL.loc[i, 'id'] = new_id.values[0]

Aprilia Sufi Subiastuti
Wiko Arif Wibowo
Wiko Arif Wibowo not found in SINTA Database


/home/matinnu/anaconda3/envs/sintamining/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
# Get Missing NIDN
missing_NIDN = ALL[pd.isnull(ALL.NIDN)]
nama = missing_NIDN.name
output_missing_NIDN = pd.DataFrame()
for i in nama:
    print(i)
    data = search_author(i)
    if data != []:
        data = pd.DataFrame(data)
        data = data[data.affiliation == affil_name_long]
        output_missing_NIDN = output_missing_NIDN.append(data)
    else:
        print(f'{i} not found in SINTA Database')
output_missing_NIDN = output_missing_NIDN[output_missing_NIDN.id.isin(ALL.id)]
#output_missing_NIDN

# Add missing NIDN
for i in ALL[ALL.NIDN.isnull()].index:
    x = ALL.loc[i, 'name'].upper()
    if output_missing_NIDN.name.isin([x]).any():
        new_id = output_missing_NIDN[output_missing_NIDN.name.isin([x])].NIDN
        ALL.loc[i, 'NIDN'] = new_id.values[0]

Wiko Arif Wibowo
Wiko Arif Wibowo not found in SINTA Database
Irfan Dwidya Prijambada
Jaka Widada
Desi Utami
Sebastian Margino
Ngadiman
Chusnul Hanim
Widya Asmara
Siti Muslimah Widyastuti
Donny Widianto
Trini Susmiati


In [11]:
# Drop null and clean
ALL = ALL[~ALL.id.isnull()]
ALL = ALL.reset_index(drop = True)
ALL.NIDN = [i.replace(' ','') for i in ALL.NIDN]
ALL.to_csv(f'../tables/{affil_name}_bio_clean.csv')

In [12]:
# get metadata from SINTA
df = pd.read_csv(f'../tables/{affil_name}_bio_clean.csv', index_col=0, dtype=str)
#sinta_get = sinta.authors(df.id)
#df2 = pd.DataFrame(sinta_get)
#df2.to_csv(f'../tables/{affil_name}_bio_clean_sinta.csv')
df2 = pd.read_csv(f'../tables/{affil_name}_bio_clean_sinta.csv', index_col=0, dtype=str)
df = df.merge(df2, left_on='id', right_on='id')#2.set_index('id', drop=False)
df['Instansi'] = affil_name_long
df.to_csv(f'../tables/{affil_name}_bio_ready.csv')